# Concrete categories and functors in DisCoPy

We show how to implement functors from Diagram to concrete categories such as monoid and tensor and how these correspond to weighted context-free grammars and tensor network models respectively. 

## Monoid
Weighted CFGs as functors in the Monoid delooping.

In [1]:
from discopy import monoidal
from discopy.monoidal import Ty
from functools import reduce
import operator

prod = lambda x: reduce(operator.mul, x, 1)

class Monoid(monoidal.Box):
    def __init__(self, m):
        self.m = m
        super().__init__(m, Ty(), Ty())

    def __repr__(self):
        return "Monoid({})".format(self.m)

    def then(self, other):
        if not isinstance(other, Monoid):
            raise ValueError
        return Monoid(self.m * other.m)

    def tensor(self, other):
        return Monoid(self.m * other.m)

    def __call__(self, *others):
        return Monoid(prod([self.m] + [other.m for other in others]))

    @staticmethod
    def id(x):
        if x != Ty():
            raise ValueError
        return Monoid(1)

assert Monoid(2) @ Monoid.id(Ty()) >> Monoid(5) @ Monoid(0.1) == Monoid(1.0)
assert Monoid(2)(Monoid(1), Monoid(4)) == Monoid(8)

A weighted CFG is a monoidal Functor from trees to Monoid.

In [2]:
from discopy.monoidal import Functor, Box, Id

class WeightedGrammar(Functor):
    def __init__(self, ar):
        ob = lambda x: Ty()
        super().__init__(ob, ar, ar_factory=Monoid)

weight = lambda box: Monoid(0.5)\
    if (box.dom, box.cod) == (Ty('N'), Ty('A', 'N')) else Monoid(1.0)

WCFG = WeightedGrammar(weight)
A = Box('A', Ty('N'), Ty('A', 'N'))
tree = A >> Id(Ty('A')) @ A
assert WCFG(tree) == Monoid(0.25)

In [3]:
from discopy.operad import from_nltk, tree2diagram
from nltk import CFG
from nltk.parse import RecursiveDescentParser
grammar = CFG.fromstring("""
S -> VP NP
NP -> D N
VP -> N V
N -> A N
V -> 'crossed'
D -> 'the'
N -> 'Moses'
A -> 'Red'
N -> 'Sea'""")

rd = RecursiveDescentParser(grammar)
parse = next(rd.parse('Moses crossed the Red Sea'.split()))
diagram = tree2diagram(from_nltk(parse))
parse2 = next(rd.parse('Moses crossed the Red Red Sea'.split()))
diagram2 = tree2diagram(from_nltk(parse2))
assert WCFG(diagram).m > WCFG(diagram2).m

## Tensor

Implementation of the category of tensors with matrix multiplication as 'then' and kronecker product as 'tensor'.
Application: Tree Tensor Networks 

## Function
Implementation of the category of Python functions with cartesian product as tensor.
Application: towards Montague moodels.

In [4]:
class Function(monoidal.Box):
    def __init__(self, inside, dom, cod):
        self.inside = inside
        name = "Function({}, {}, {})".format(inside, dom, cod)
        super().__init__(name, dom, cod)

    def then(self, other):
        inside = lambda *xs: other(*tuple(self(*xs)))
        return Function(inside, self.dom, other.cod)

    def tensor(self, other):
        def inside(*xs):
            left, right = xs[:len(self.dom)], xs[len(self.dom):]
            result = tuple(self(*left)) + tuple(other(*right))
            return (result[0], ) if len(self.cod @ other.cod) == 1 else result
        return Function(inside, self.dom @ other.dom, self.cod @ other.cod)

    def __call__(self, *xs): return self.inside(*xs)

    @staticmethod
    def id(x):
        return Function(lambda *xs: xs, x, x)

    @staticmethod
    def copy(x):
        return Function(lambda *xs: (*xs, *xs), x, x @ x)
    
    @staticmethod
    def delete(x):
        return Function(lambda *xs: (), x, Ty())
    
    @staticmethod
    def swap(x, y):
        return Function(lambda x0, y0: (y0, x0), x @ y, y @ x)

In [5]:
from discopy.monoidal import Ty, Id, Box, Functor

X = Ty('X')

two, three, five = Box('two', Ty(), X), Box('three', Ty(), X), Box('five', Ty(), X)
plus, is_ = Box('plus', X @ X, X), Box('is', X @ X, X)
sentence = two @ three @ five >> plus @ Id(X) >> is_

number = lambda y: Function(lambda: (y, ), Ty(), X)
add = Function(lambda x, y: (x + y,), X @ X, X)
is_equal = Function(lambda x, y: (x == y, ), X @ X, X)

ob = lambda x: x
ar = {two: number(2), three: number(3), five: number(5),
      is_: is_equal, plus: add}
F = Functor(ob, ar, ob_factory=Ty, ar_factory=Function)
F(sentence)()

(True,)

In [6]:
copy = Function.copy(X)
delete = Function.delete(X)
I = Function.id(X)
swap = Function.swap(X, X)

assert (copy >> copy @ I)(54) == (copy >> I @ copy)(54)
assert (copy >> delete @ I)(46) == (copy >> I @ delete)(46)
assert (copy >> swap)('was my number') == (copy)('was my number')